In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.dates as mdates

# -------------------------
# KLASÖR VE DOSYA YAPISI
# -------------------------
INPUT_FILE = '/content/127 1.xls'
OUTPUT_DIR = 'outputs'

# -------------------------
# ANA AKIŞ
# -------------------------
def main():
    # 1. Çıktı klasörünü oluştur
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # 2. Veriyi yükle
    df = pd.read_excel(INPUT_FILE)
    df['Tarih'] = pd.to_datetime(df['Tarih'])

    # 3. Özellik mühendisliği
    df['Yil'] = df['Tarih'].dt.year
    df['Ay'] = df['Tarih'].dt.month
    df['Gun'] = df['Tarih'].dt.day
    df['Hafta'] = df['Tarih'].dt.isocalendar().week
    df['HaftaninGunu'] = df['Tarih'].dt.weekday
    df['GunIsmi'] = df['Tarih'].dt.day_name()
    df['HaftaSonu'] = df['HaftaninGunu'].apply(lambda x: 1 if x >= 5 else 0)

    def get_season(month):
        if month in [12, 1, 2]:
            return 'Kış'
        elif month in [3, 4, 5]:
            return 'İlkbahar'
        elif month in [6, 7, 8]:
            return 'Yaz'
        else:
            return 'Sonbahar'

    df['Mevsim'] = df['Ay'].apply(get_season)

    # 4. Önbakı ve kaydetme
    print("Veri Seti Boyutu:", df.shape)
    print(df.dtypes)
    print(df.describe())
    print(df.head())
    df.to_csv(os.path.join(OUTPUT_DIR, 'preprocessed_data.csv'), index=False)
    print(f"Ön işlenmiş veri kaydedildi: {os.path.join(OUTPUT_DIR, 'preprocessed_data.csv')}")

    # 5. Aylık Ortalama Ciro Trendi
    df_aylik = df.resample('M', on='Tarih')['Ciro'].mean().reset_index()
    plt.figure(figsize=(15, 6))
    plt.plot(df_aylik['Tarih'], df_aylik['Ciro'], alpha=0.7)
    plt.title('Aylık Ortalama Ciro Trendi')
    plt.xlabel('Tarih')
    plt.ylabel('Ortalama Ciro (TL)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'aylik_ortalama_ciro_trendi.png'))
    plt.close()

    # 6. Yıllara Göre Aylık Ortalama Ciro
    yillik_aylik_ciro = df.groupby(['Yil', 'Ay'])['Ciro'].mean().unstack()
    plt.figure(figsize=(15, 8))
    for yil in yillik_aylik_ciro.index:
        plt.plot(yillik_aylik_ciro.columns, yillik_aylik_ciro.loc[yil], marker='o', label=yil)
    plt.title('Yıllara Göre Aylık Ciro Trendi')
    plt.xlabel('Ay')
    plt.ylabel('Ortalama Ciro (TL)')
    plt.xticks(range(1, 13), ['Ocak','Şubat','Mart','Nisan','Mayıs','Haziran','Temmuz','Ağustos','Eylül','Ekim','Kasım','Aralık'], rotation=45)
    plt.legend(title='Yıl')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'yillara_gore_aylik_ortalama_ciro.png'))
    plt.close()

    # 7. Yıllara Göre Aylık Toplam Ciro
    yillik_aylik_sum = df.groupby(['Yil','Ay'])['Ciro'].sum().unstack()
    yillik_aylik_sum.to_csv(os.path.join(OUTPUT_DIR, 'yillara_gore_aylik_toplam_ciro.csv'))

    # 8. Mevsimlere Göre Ortalama Ciro
    season_order = ['İlkbahar','Yaz','Sonbahar','Kış']
    seasonal_avg = df.groupby('Mevsim')['Ciro'].mean().reindex(season_order)
    plt.figure(figsize=(12,6))
    seasonal_avg.plot(kind='bar')
    plt.title('Mevsimlere Göre Ortalama Ciro')
    plt.xlabel('Mevsim')
    plt.ylabel('Ortalama Ciro (TL)')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'mevsimlere_gore_ortalama_ciro.png'))
    plt.close()

    # 9. Hafta İçi vs Hafta Sonu
    haftasonu = df.groupby('HaftaSonu')['Ciro'].mean()
    plt.figure(figsize=(8,6))
    haftasonu.plot(kind='bar')
    plt.title('Hafta İçi vs Hafta Sonu Ortalama Ciro')
    plt.xlabel('HaftaSonu')
    plt.ylabel('Ortalama Ciro (TL)')
    plt.xticks([0,1], ['Hafta İçi','Hafta Sonu'], rotation=0)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'haftaici_vs_haftasonu_ciro.png'))
    plt.close()

    # 10. Aylık Trend (Yıl-Ay)
    monthly_avg = df.groupby(['Yil','Ay'])['Ciro'].mean().reset_index()
    monthly_avg['YilAy'] = monthly_avg['Yil'].astype(str) + '-' + monthly_avg['Ay'].astype(str).str.zfill(2)
    plt.figure(figsize=(15,6))
    plt.plot(monthly_avg['YilAy'], monthly_avg['Ciro'], marker='o')
    plt.title('Aylık Ortalama Ciro Trendi (Yıl-Ay)')
    plt.xlabel('Yıl-Ay')
    plt.ylabel('Ortalama Ciro (TL)')
    plt.xticks(rotation=90)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'aylik_ortalama_trend_yilay.png'))
    plt.close()

    # 11. Bayram Etkisi
    bayram = df[df['BayramEtkisi']>0].groupby('BayramEtkisi')['Ciro'].mean()
    if not bayram.empty:
        plt.figure(figsize=(10,6))
        bayram.plot(kind='bar')
        plt.title('Bayram Etkisinin Ortalama Ciroya Etkisi')
        plt.xlabel('Bayram Etki Seviyesi')
        plt.ylabel('Ortalama Ciro (TL)')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_DIR, 'bayram_etkisi_ciro.png'))
        plt.close()

    # 12. Korelasyon Matrisi
    corr_cols = ['Ciro','HaftaSonu','BayramEtkisi','Ay','Gun','Hafta','HaftaninGunu']
    plt.figure(figsize=(10,8))
    sns.heatmap(df[corr_cols].corr(), annot=True, fmt='.2f', linewidths=0.5)
    plt.title('Korelasyon Matrisi')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'korelasyon_matrisi.png'))
    plt.close()

    # 13. Yıllara Göre Aylık Karşılaştırma
    yillar = df['Yil'].unique()
    if len(yillar)>1:
        pivot = df.pivot_table(index='Ay', columns='Yil', values='Ciro', aggfunc='mean')
        plt.figure(figsize=(15,8))
        pivot.plot(kind='bar')
        plt.title('Yıllara Göre Aylık Ortalama Ciro')
        plt.xlabel('Ay')
        plt.ylabel('Ortalama Ciro (TL)')
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_DIR, 'yillara_gore_aylik_ciro.png'))
        plt.close()

    # 14. Günlere Göre Ortalama Ciro
    day_order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    daily_avg = df.groupby('GunIsmi')['Ciro'].mean().reindex(day_order)
    tr_days = {'Monday':'Pazartesi','Tuesday':'Salı','Wednesday':'Çarşamba','Thursday':'Perşembe','Friday':'Cuma','Saturday':'Cumartesi','Sunday':'Pazar'}
    plt.figure(figsize=(12,6))
    daily_avg.plot(kind='bar')
    plt.title('Günlere Göre Ortalama Ciro')
    plt.xlabel('Gün')
    plt.ylabel('Ortalama Ciro (TL)')
    plt.xticks(range(len(day_order)), [tr_days[d] for d in day_order], rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'gunlere_gore_ortalama_ciro.png'))
    plt.close()

    # 15. Bayram Etkisi Ciro Barplot
    bay_ciro = df.groupby('BayramEtkisi')['Ciro'].mean()
    plt.figure(figsize=(8,6))
    sns.barplot(x=bay_ciro.index, y=bay_ciro.values)
    plt.title('Bayram Etkisinin Ciroya Etkisi')
    plt.xlabel('Bayram Etkisi')
    plt.ylabel('Ortalama Ciro (TL)')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'bayram_etkisi_barplot.png'))
    plt.close()

    print("\n Tüm ön işleme ve görselleştirme adımları tamamlandı. Çıktılar 'outputs/' klasöründe.")

if __name__ == '__main__':
    main()


Veri Seti Boyutu: (1119, 13)
Tarih               datetime64[ns]
DepoRef                      int64
HaftaSonu                    int64
BayramEtkisi               float64
CalismaGunSayisi             int64
Ciro                       float64
Yil                          int32
Ay                           int32
Gun                          int32
Hafta                       UInt32
HaftaninGunu                 int32
GunIsmi                     object
Mevsim                      object
dtype: object
                               Tarih  DepoRef    HaftaSonu  BayramEtkisi  \
count                           1119   1119.0  1119.000000   1119.000000   
mean   2023-07-18 01:27:30.402144768    127.0     0.285970      0.059875   
min              2022-01-01 00:00:00    127.0     0.000000      0.000000   
25%              2022-10-07 12:00:00    127.0     0.000000      0.000000   
50%              2023-07-15 00:00:00    127.0     0.000000      0.000000   
75%              2024-04-19 12:00:00    127.0 

<ipython-input-4-2715a4d109a5>:56: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_aylik = df.resample('M', on='Tarih')['Ciro'].mean().reset_index()



 Tüm ön işleme ve görselleştirme adımları tamamlandı. Çıktılar 'outputs/' klasöründe.


<Figure size 1500x800 with 0 Axes>